In [1]:
import json
from transformers import BertTokenizer, BertForQuestionAnswering


with open('./data/squad/dev.json', 'r') as f:
    squad = json.load(f)

In [4]:
from haystack.document_stores import ElasticsearchDocumentStore
import requests

res = requests.get('http://localhost:9200/_cluster/health',verify=False)

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [5]:
document_store = ElasticsearchDocumentStore(host = 'localhost', username ='elastic', password = 'Ci=SX6m=gTUlH8vDfLfa', index = 'squad_docs')

In [7]:
squad_docs = []

for sample in squad:
    squad_docs.append({
        'content': sample['context'],
    })
squad_docs[:2]

[{'content': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'},
 {'content': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norsem

In [8]:
document_store.write_documents(squad_docs)

In [11]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store)

In [12]:
query = "what century did the Normans first gain their separeted identidy"

retriever.retrieve(query)

[<Document: {'content': 'In the visual arts, the Normans did not have the rich and distinctive traditions of the cultures they conquered. However, in the early 11th century the dukes began a programme of church reform, encouraging the Cluniac reform of monasteries and patronising intellectual pursuits, especially the proliferation of scriptoria and the reconstitution of a compilation of lost illuminated manuscripts. The church was utilised by the dukes as a unifying force for their disparate duchy. The chief monasteries taking part in this "renaissance" of Norman art and scholarship were Mont-Saint-Michel, Fécamp, Jumièges, Bec, Saint-Ouen, Saint-Evroul, and Saint-Wandrille. These centres were in contact with the so-called "Winchester school", which channeled a pure Carolingian artistic tradition to Normandy. In the final decade of the 11th and first of the 12th century, Normandy experienced a golden age of illustrated manuscripts, but it was brief and the major scriptoria of Normandy 

In [13]:
res = requests.post('http://localhost:9200/squad_docs/_delete_by_query',
                   json = {'query':{'match_all':{}}})

In [15]:
res = requests.get('http://localhost:9200/squad_docs/_count')
res.json()


{'count': 0,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [19]:
contexts = [sample['context'] for sample in squad]
contexts = list(set(contexts))


In [21]:
squad_docs = [{'content': sample} for sample in contexts]
document_store.write_documents(squad_docs)
retriever = BM25Retriever(document_store)

In [22]:
retriever.retrieve(query)

[<Document: {'content': 'In the visual arts, the Normans did not have the rich and distinctive traditions of the cultures they conquered. However, in the early 11th century the dukes began a programme of church reform, encouraging the Cluniac reform of monasteries and patronising intellectual pursuits, especially the proliferation of scriptoria and the reconstitution of a compilation of lost illuminated manuscripts. The church was utilised by the dukes as a unifying force for their disparate duchy. The chief monasteries taking part in this "renaissance" of Norman art and scholarship were Mont-Saint-Michel, Fécamp, Jumièges, Bec, Saint-Ouen, Saint-Evroul, and Saint-Wandrille. These centres were in contact with the so-called "Winchester school", which channeled a pure Carolingian artistic tradition to Normandy. In the final decade of the 11th and first of the 12th century, Normandy experienced a golden age of illustrated manuscripts, but it was brief and the major scriptoria of Normandy 